# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [29]:
# Write your code below.

import os
from dotenv import load_dotenv
load_dotenv()

print(os.environ.get("PRICE_DATA"))

../../05_src/data/prices/


In [3]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [31]:
from glob import glob
import os

price_data_dir = os.environ["PRICE_DATA"]

parquet_files = glob(os.path.join(price_data_dir, "**", "*.parquet"), recursive=True)

print("Number of parquet files:", len(parquet_files))
parquet_files[:5]




Number of parquet files: 3014


['../../05_src/data/prices/BKTI/BKTI_2012/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2012/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2014/part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [45]:
import dask.dataframe as dd

# make sure Date is datetime
ddf["Date"] = dd.to_datetime(ddf["Date"])

def add_features(pdf):
    # pandas-only inside each partition
    pdf = pdf.sort_values(["ticker", "Date"])

    pdf["Close_lag_1"] = pdf.groupby("ticker")["Close"].shift(1)
    pdf["Adj_Close_lag_1"] = pdf.groupby("ticker")["Adj Close"].shift(1)

    pdf["returns"] = (pdf["Close"] / pdf["Close_lag_1"]) - 1
    pdf["hi_lo_range"] = pdf["High"] - pdf["Low"]

    return pdf

dd_feat = ddf.map_partitions(add_features, meta=ddf._meta.assign(
    Close_lag_1="f8",
    Adj_Close_lag_1="f8",
    returns="f8",
    hi_lo_range="f8"
))




In [46]:
# how many NaNs in Close_lag_1 per ticker?
nan_counts = dd_feat.groupby("ticker")["Close_lag_1"].apply(
    lambda s: s.isna().sum(),
    meta=("Close_lag_1", "i8")
)
nan_counts.compute().describe()


count    90.000000
mean     33.588889
std      22.626868
min       2.000000
25%      16.000000
50%      29.000000
75%      49.500000
max      98.000000
Name: Close_lag_1, dtype: float64

Note: The number of missing values in `Close_lag_1` is greater than one for most tickers.  
This is expected in Dask because lag features are computed independently within each partition, and each partition introduces an initial missing value.


In [35]:
cols = ["Date","ticker","Close","Close_lag_1","returns","High","Low","hi_lo_range"]
dd_feat[dd_feat["ticker"] == "A"][cols].head(10, compute=True)


,Date,ticker,Close,Close_lag_1,returns,High,Low,hi_lo_range
38739,1999-11-18,A,31.473534,NaN,NaN,35.765381,28.612303,7.153078
38740,1999-11-19,A,28.880543,31.473534,-0.082386,30.758226,28.478184,2.280043
38741,1999-11-22,A,31.473534,28.880543,0.089783,31.473534,28.657009,2.816525
38742,1999-11-23,A,28.612303,31.473534,-0.090909,31.205294,28.612303,2.592991
38743,1999-11-24,A,29.372318,28.612303,0.026563,29.998211,28.612303,1.385908
38744,1999-11-26,A,29.461731,29.372318,0.003044,29.685265,29.148785,0.536480
38745,1999-11-29,A,30.132332,29.461731,0.022762,30.355865,29.014664,1.341202
38746,1999-11-30,A,30.177038,30.132332,0.001484,30.713520,29.282904,1.430616
38747,1999-12-01,A,30.713520,30.177038,0.017778,31.071173,29.953505,1.117668
38748,1999-12-02,A,31.562946,30.713520,0.027656,32.188843,30.892345,1.296497


In [37]:
cols = ["Date","ticker","Adj Close","Adj_Close_lag_1"]
dd_feat[dd_feat["ticker"]=="A"][cols].head(10, compute=True)


,Date,ticker,Adj Close,Adj_Close_lag_1
38739,1999-11-18,A,27.068665,NaN
38740,1999-11-19,A,24.838577,27.068665
38741,1999-11-22,A,27.068665,24.838577
38742,1999-11-23,A,24.607880,27.068665
38743,1999-11-24,A,25.261524,24.607880
38744,1999-11-26,A,25.338428,25.261524
38745,1999-11-29,A,25.915169,25.338428
38746,1999-11-30,A,25.953619,25.915169
38747,1999-12-01,A,26.415012,25.953619
38748,1999-12-02,A,27.145563,26.415012


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [38]:
# Convert Dask DataFrame to pandas DataFrame
pdf_feat = dd_feat.compute()


In [39]:
pdf_feat = pdf_feat.sort_values(["ticker", "Date"])


In [40]:
pdf_feat["returns_ma_10"] = (
    pdf_feat
    .groupby("ticker")["returns"]
    .rolling(10)
    .mean()
    .reset_index(level=0, drop=True)
)


In [41]:
cols = ["Date", "ticker", "returns", "returns_ma_10"]
pdf_feat[pdf_feat["ticker"] == "A"][cols].head(15)


,Date,ticker,returns,returns_ma_10
38739,1999-11-18,A,NaN,NaN
38740,1999-11-19,A,-0.082386,NaN
38741,1999-11-22,A,0.089783,NaN
38742,1999-11-23,A,-0.090909,NaN
38743,1999-11-24,A,0.026563,NaN
38744,1999-11-26,A,0.003044,NaN
38745,1999-11-29,A,0.022762,NaN
38746,1999-11-30,A,0.001484,NaN
38747,1999-12-01,A,0.017778,NaN
38748,1999-12-02,A,0.027656,NaN


In [42]:
pdf_feat[pdf_feat["ticker"]=="A"][["Date","returns"]].iloc[1:11]["returns"].mean()


np.float64(0.002427277802136263)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?


Was it necessary to convert to pandas to calculate the moving average return?
No, not strictly. Dask can also compute rolling averages. The conversion to pandas was done mainly because the assignment explicitly asked for it, not because the calculation required pandas.


Would it have been better to do it in Dask? Why?
In general, yes,especially for large datasets. Dask is more memory-efficient and scalable. However, rolling windows in Dask are more complex because of partition boundaries, so pandas is simpler and safer for this assignment.

(1 pt)

References (APA ):


Dask Developers. (n.d.). Dask DataFrame documentation. Retrieved January 19, 2026, from https://docs.dask.org/en/stable/dataframe.html
Dask Developers. (n.d.). dask.dataframe.read_parquet. Retrieved January 19, 2026, from https://docs.dask.org/en/stable/generated/dask.dataframe.read_parquet.html
pandas Development Team. (n.d.). pandas.DataFrame.rolling. Retrieved January 19, 2026, from https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html
Python Software Foundation. (n.d.). glob — Unix style pathname pattern expansion. In Python documentation. Retrieved January 19, 2026, from https://docs.python.org/3/library/glob.html
Saurabh Kumar. (n.d.). python-dotenv documentation. Retrieved January 19, 2026, from https://saurabh-kumar.com/python-dotenv/

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.